In [ ]:
import iris_pipeline
iris_pipeline.monkeypatch_jwst_datamodels()

import numpy as np

In [ ]:
iris_pipeline.__file__

The purpose of this notebook is to create a dataset with a full frame and relative subarrays

## Prepare the raw science frame

* Load raw science frame
* Add `SUBARR_MAP` extension

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from test_utils import get_data_from_url
raw_science_filename = get_data_from_url("17903858")
im = iris_pipeline.datamodels.IRISImageModel(raw_science_filename)

In [ ]:
s1 = 300
s2 = 100
subarray_maps_metadata = []
# square subarray
subarray_maps_metadata.append({"xstart":2048-s1//2, "ystart":2048-s1//2, "xsize":s1, "ysize":s1})  
# test a rectangular subarray
subarray_maps_metadata.append({"xstart":1040-s2//2, "ystart":1040-s2//2, "xsize":s2, "ysize":s2//2})  

In [ ]:
plt.imshow(im.data);

In [ ]:
def set_subarray_mask(mask_array, subarray_index, xstart, ystart, xsize, ysize):
    xstart = xstart - 1
    ystart = ystart - 1
    mask_array[ystart:ystart+ysize, xstart:xstart+xsize] =  subarray_index                 

In [ ]:
im.subarr_map = np.zeros_like(im.data)

In [ ]:
for i, shape in enumerate(subarray_maps_metadata):
    set_subarray_mask(im.subarr_map, subarray_index=i+1, **shape)

In [ ]:
plt.imshow(im.data[im.subarr_map==1].reshape((s1,s1)));

In [ ]:
plt.imshow(im.data[im.subarr_map==2].reshape((s2//2,-1)));

In [ ]:
plt.imshow(im.subarr_map);

In [ ]:
im.data[im.subarr_map != 0] = np.nan

In [ ]:
im.write("data/raw_science_frame_sci_with_subarrays.fits")

## Run the image2 pipeline

In [ ]:
pipeline = iris_pipeline.Image2Pipeline()

output = pipeline.call("data/raw_science_frame_sci_with_subarrays.fits",
             config_file="data/image2_iris.cfg")[0]

## Check the output has correct metadata

In [ ]:
for each_parsed, each_input in zip(output.meta.subarray_map, subarray_maps_metadata):
    assert each_parsed.instance == each_input